
Naver Land Crawler (NAVER ONLY)
- 입력: 사용자 키워드(예: "강남구 전세 84", "마포구 매매", "역삼동 월세")
- 동작: 키워드 → 파라미터 해석 → clusterList → articleList(그룹별 페이지네이션) → 정규화 → CSV 저장
- 주의: 네이버 부동산 API는 '비공식'입니다. 헤더/리퍼러/호출간 지연을 반드시 지키세요.

검증 근거:
- /cluster/ajax/articleList 호출 예시 및 파라미터: 커뮤니티/블로그 실측 기록 다수  # :contentReference[oaicite:2]{index=2}
- /cluster/clusterList → 그룹(lgeo, count) 선행 후 articleList로 매물 조회 절차  # :contentReference[oaicite:3]{index=3}
- 지도 기반 모바일 페이지 경로와 실제 호출 흐름(지도 뷰/줌/좌표)  # :contentReference[oaicite:4]{index=4}


In [10]:
from __future__ import annotations
import os
import re
import csv
import math
import time
import json
import random
from typing import List, Dict, Any, Tuple, Optional

import requests


# 1. 환경 상수

In [11]:
USER_AGENT = (
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)

# 네이버 모바일 지도 XHR 엔드포인트 (비공식, 실측)
NAVER_CLUSTER_LIST = "https://m.land.naver.com/cluster/clusterList"          # 1단계: 그룹(버블) 조회  # :contentReference[oaicite:5]{index=5}
NAVER_ARTICLE_LIST = "https://m.land.naver.com/cluster/ajax/articleList"     # 2단계: 매물 리스트     # :contentReference[oaicite:6]{index=6}

# 거래/매물 타입 매핑
TRADE_MAP = {"매매": "A1", "전세": "B1", "월세": "B2"}        # tradTpCd
RLET_MAP  = {"아파트": "APT", "오피스텔": "OPST", "원룸": "GM"}  # rletTpCd

# 디폴트 지도 상태(예: 강남역 부근)
DEFAULT_LAT, DEFAULT_LON, DEFAULT_Z = 37.4979, 127.0276, 17


# 2. 유틸 함수 세팅

In [12]:
def make_session() -> requests.Session:
    s = requests.Session()
    s.headers.update({
        "User-Agent": USER_AGENT,
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://m.land.naver.com/",
    })
    return s

def jitter(min_s=0.8, max_s=1.8):
    time.sleep(random.uniform(min_s, max_s))

def parse_int(s: Any) -> Optional[int]:
    try:
        return int(str(s).replace(",", "").strip())
    except Exception:
        return None

def price_to_manwon(s: Optional[str]) -> Optional[int]:
    """
    네이버 priceString/문자형 가격을 '만원' 단위 정수로 변환.
    예: '12억 3,000' -> 123000, '3,500' -> 3500, '전세 6억' -> 60000
    """
    if not s:
        return None
    t = re.sub(r"[^\d억만천백십 ]", "", s)
    eok = 0
    man = 0
    m = re.search(r"(\d+)\s*억", t)
    if m: eok = int(m.group(1))
    m = re.search(r"(\d+)\s*만", t)
    if m: man = int(m.group(1))
    if ("억" not in t) and ("만" not in t):
        digits = re.sub(r"\D", "", t)
        if digits:
            man = int(digits)
    return eok * 10000 + man

def normalize_article_response(js: Any) -> List[Dict[str, Any]]:
    """
    articleList 응답 래핑은 상황에 따라 다름:
    - 최상위 dict에 'body'가 list
    - 'body'가 dict이고 그 안에 'articles' 또는 'list'
    - 최상위가 list
    → 안전 분기 처리
    """
    if isinstance(js, list):
        return js
    if isinstance(js, dict):
        body = js.get("body")
        if isinstance(body, list):
            return body
        if isinstance(body, dict):
            arr = body.get("articles") or body.get("list")
            if isinstance(arr, list):
                return arr
        arr = js.get("articles")
        if isinstance(arr, list):
            return arr
    return []

def parse_keyword(q: str) -> Dict[str, Any]:
    """
    매우 단순한 규칙 기반 파서:
    - 거래유형(없으면 전체 A1:B1:B2)
    - 매물유형(없으면 아파트)
    - 면적 힌트(숫자 2~3자리)
    """
    q = q.strip()
    trad = None
    for k, v in TRADE_MAP.items():
        if k in q:
            trad = v
            break
    rlet = "APT"
    for k, v in RLET_MAP.items():
        if k in q:
            rlet = v
            break
    m = re.search(r"(\d{2,3})\s?㎡?", q)
    area_hint = int(m.group(1)) if m else None
    # 지역 토큰은 여기서는 그대로 문자열로 보유(좌표/코드 매핑은 별도 설계)
    return {"tradTpCd": trad or "A1:B1:B2", "rletTpCd": rlet, "area_hint": area_hint, "raw": q}


# 3-1. 클러스터 조회 -> 그룹(버블) 목록

In [13]:
def fetch_cluster_groups(
    sess: requests.Session,
    *,
    rletTpCd: str,
    tradTpCd: str,
    lat: float,
    lon: float,
    z: int,
    cortarNo: Optional[str] = None,
    bbox: Optional[Dict[str, float]] = None,
) -> List[Dict[str, Any]]:
    """
    /cluster/clusterList 호출해 지도상 그룹 목록(data.ARTICLE[]) 수집.
    - 필수: view=atcl, rletTpCd, tradTpCd, z, lat, lon
    - 선택: cortarNo(행정/법정동 코드), bbox(지도 사각영역: btm,lft,top,rgt)
    """
    q = {
        "view": "atcl",
        "rletTpCd": rletTpCd,
        "tradTpCd": tradTpCd,
        "z": str(z),
        "lat": str(lat),
        "lon": str(lon),
    }
    if cortarNo:
        q["cortarNo"] = str(cortarNo)
    if bbox:
        q.update({k: str(v) for k, v in bbox.items()})
    r = sess.get(NAVER_CLUSTER_LIST, params=q, timeout=20)
    r.raise_for_status()
    js = r.json()
    groups = (js.get("data") or {}).get("ARTICLE") or []
    return groups  # 각 원소: {lgeo, count, z, lat, lon, ...}


# 3-2. 그룹별 페이징 처리

In [14]:
def fetch_articles_by_group(
    sess: requests.Session,
    group: Dict[str, Any],
    *,
    rletTpCd: str,
    tradTpCd: str,
    cortarNo: Optional[str] = None,
    page_size: int = 20,
    max_pages: Optional[int] = None,
) -> List[Dict[str, Any]]:
    """
    /cluster/ajax/articleList 호출.
    필수 파라미터:
      - itemId(=lgeo), totCnt(=count), page(1..), z/lat/lon(그룹에서 받은 값)
      - rletTpCd, tradTpCd
    """
    lgeo  = group.get("lgeo")
    count = int(group.get("count", 0))
    if not lgeo or count <= 0:
        return []

    z   = group.get("z")
    lat = group.get("lat")
    lon = group.get("lon")

    pages = max(1, math.ceil(count / page_size))
    if max_pages:
        pages = min(pages, max_pages)

    results: List[Dict[str, Any]] = []
    for page in range(1, pages + 1):
        q = {
            "itemId": lgeo,
            "lgeo": lgeo,
            "totCnt": str(count),
            "z": str(z),
            "lat": str(lat),
            "lon": str(lon),
            "rletTpCd": rletTpCd,
            "tradTpCd": tradTpCd,
            "page": str(page),
            "showR0": "",  # 관측상 종종 포함됨
        }
        if cortarNo:
            q["cortarNo"] = str(cortarNo)

        r = sess.get(NAVER_ARTICLE_LIST, params=q, timeout=20)
        r.raise_for_status()
        arr = normalize_article_response(r.json())
        if not arr:
            # 키 구조가 바뀌었을 수 있으니 진단 로그(필요 시)
            # print("Empty page; keys:", r.json().keys() if isinstance(r.json(), dict) else type(r.json()))
            pass
        results.extend(arr)
        jitter()
    return results

# 4. 정규화

In [15]:
def normalize_article_row(a: Dict[str, Any]) -> Dict[str, Any]:
    """
    필드명이 종종 바뀌므로(or atclNm vs articleName 등) OR 체인으로 보완.
    금액은 '만원' 단위 정수로 정규화.
    """
    atcl_no = a.get("atclNo") or a.get("articleNo")
    name    = a.get("atclNm") or a.get("articleName")
    price_s = a.get("prc") or a.get("priceString")
    floor   = a.get("flrInfo") or a.get("floorInfo")
    realtor = a.get("rltrNm") or a.get("realtorName")
    area1   = a.get("spc1") or a.get("area1")  # 공급/전용 혼재 → 참고용
    area2   = a.get("spc2") or a.get("area2")
    addr    = a.get("addr") or a.get("roadAddress") or a.get("address")

    price_man = price_to_manwon(price_s)

    return {
        "atcl_no": atcl_no,
        "name": name,
        "price_str": price_s,
        "price_man": price_man,    # 만원 단위
        "floor": floor,
        "realtor": realtor,
        "area1": area1,
        "area2": area2,
        "addr": addr,
        "raw": a,
    }

In [16]:
def filter_by_area_hint(rows: List[Dict[str, Any]], area_hint: Optional[int]) -> List[Dict[str, Any]]:
    if not area_hint:
        return rows
    out = []
    for r in rows:
        # area1/area2 중 숫자 추출해 근사 매칭(±1~2㎡ 여유)
        def _to_float(x):
            try:
                return float(str(x).replace(",", "").replace("㎡", "").strip())
            except Exception:
                return None
        cand = []
        for key in ("area1", "area2"):
            v = _to_float(r.get(key))
            if v is not None:
                cand.append(v)
        if not cand:
            out.append(r)  # 면적 미표기 → 일단 통과
            continue
        if any(abs(v - area_hint) <= 2 for v in cand):
            out.append(r)
    return out

# 5. 실행

In [17]:
def crawl_naver_by_keyword(
    keyword: str,
    *,
    lat: float = DEFAULT_LAT,
    lon: float = DEFAULT_LON,
    z: int = DEFAULT_Z,
    cortarNo: Optional[str] = None,   # 있으면 지역 제한 ↑ (법정/행정동 코드)
    bbox: Optional[Dict[str, float]] = None,  # 지도 사각형이 있으면 정확도/회수율 ↑
    max_groups: Optional[int] = None,         # 그룹 수가 많을 때 제한
    max_pages_per_group: Optional[int] = None # 그룹 내 페이지 제한
) -> List[Dict[str, Any]]:
    """
    좌표/줌+선택적 cortarNo/bbox를 기반으로, 키워드에서 tradTpCd/rletTpCd/면적힌트를 받아 수집.
    """
    parsed = parse_keyword(keyword)
    tradTpCd = parsed["tradTpCd"]
    rletTpCd = parsed["rletTpCd"]
    area_hint = parsed["area_hint"]

    sess = make_session()

    # 1) 그룹 수집
    groups = fetch_cluster_groups(
        sess,
        rletTpCd=rletTpCd,
        tradTpCd=tradTpCd,
        lat=lat, lon=lon, z=z,
        cortarNo=cortarNo,
        bbox=bbox,
    )
    if not groups:
        return []

    if max_groups:
        groups = groups[:max_groups]

    # 2) 그룹별 매물
    articles: List[Dict[str, Any]] = []
    for g in groups:
        items = fetch_articles_by_group(
            sess, g,
            rletTpCd=rletTpCd, tradTpCd=tradTpCd,
            cortarNo=cortarNo,
            max_pages=max_pages_per_group,
        )
        articles.extend(items)
        jitter()

    # 3) 정규화
    rows = [normalize_article_row(a) for a in articles]

    # 4) 면적 힌트 필터(선택)
    rows = filter_by_area_hint(rows, area_hint)

    # atcl_no 중복 제거
    uniq = {}
    for r in rows:
        key = r.get("atcl_no") or id(r)
        if key not in uniq:
            uniq[key] = r
    return list(uniq.values())

def save_csv(rows: List[Dict[str, Any]], path: str):
    if not rows:
        print("No data; skip write.")
        return
    cols = ["atcl_no", "name", "price_str", "price_man", "floor", "realtor", "area1", "area2", "addr"]
    with open(path, "w", newline="", encoding="utf-8-sig") as f:
        wr = csv.DictWriter(f, fieldnames=cols)
        wr.writeheader()
        for r in rows:
            wr.writerow({k: r.get(k) for k in cols})

In [18]:
# 예시 실행
# - 키워드: 거래/매물타입/면적 힌트를 파싱(없으면 기본값 적용)
# - 좌표/줌: 현재 보고 싶은 지도 중심(예: 강남역)
# - cortarNo: 지역 제한(선택). 없으면 좌표/줌/bbox만으로도 동작.
KEYWORD = os.getenv("NKEYWORD", "강남구 전세 84")
LAT = float(os.getenv("NLAT", DEFAULT_LAT))
LON = float(os.getenv("NLON", DEFAULT_LON))
Z   = int(os.getenv("NZ", DEFAULT_Z))
CORTAR = os.getenv("NCORTAR", "") or None

# 필요 시 지도 사각형(Bounding Box)도 주면 회수율↑ (좌표 4개 모두 문자열/숫자 OK)
BBOX = None
# BBOX = {"btm": 37.49, "lft": 127.01, "top": 37.51, "rgt": 127.04}

rows = crawl_naver_by_keyword(
    KEYWORD,
    lat=LAT, lon=LON, z=Z,
    cortarNo=CORTAR,
    bbox=BBOX,
    max_groups=int(os.getenv("NMAX_GROUPS", "20")),          # 과도 수집 방지
    max_pages_per_group=int(os.getenv("NMAX_PAGES", "3")),   # 그룹당 3페이지(약 60건)
)
print(f"collected: {len(rows)} rows")
save_csv(rows, "naver_listings.csv")
print("saved -> naver_listings.csv")

collected: 0 rows
No data; skip write.
saved -> naver_listings.csv
